In [ ]:
# importing necessary libraries
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
# Create the environment
env = make_vec_env('Walker2d-v4', n_envs=1)

# Define the model
model = PPO(
    "MlpPolicy",
    env,
    learning_rate = 3e-4,
    n_steps = 4096,
    batch_size = 64,
    n_epochs = 10,
    gamma = 0.99,
    gae_lambda = 0.95,
    clip_range = 0.2,
    ent_coef = 0.01,
    vf_coef = 0.5,
    max_grad_norm = 0.5,
    verbose=1
)

model.learn(total_timesteps=4000000)

In [ ]:
# save the model
model.save("ppo_walker2d")

In [ ]:
# Evaluate the model
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=25)
print(f'Reward : {mean_reward} +/- {std_reward}')

In [ ]:
# Recording the video
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

env_id = 'Walker2d-v4'
video_folder = './videos'
video_length = 1000

vec_env = DummyVecEnv([lambda: gym.make(env_id, render_mode='rgb_array')])
obs = vec_env.reset()

vec_env = VecVideoRecorder(vec_env, video_folder=video_folder, record_video_trigger=lambda x: x%1000 == 0, 
                           video_length=video_length, name_prefix=f'Walker2d-v4-{video_length}')

vec_env.reset()
for i in range(video_length + 1):
    action , _states = model.predict(obs)
    obs, rewards, done, info = vec_env.step(action)

vec_env.close()


In [ ]:
# Uploading model to hugging face
import gymnasium as gym
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env

from huggingface_sb3 import package_to_hub

model_name = "ppo-Walker2d-v4"

# Define a repo_id
repo_id = 'Amankankriya/ppo-Walker2d-v4'

# Define the name of the environment
env_id = 'Walker2d-v4'

# Create the evaluation env and set the render_mode="rgb_array"
eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])


# Define the model architecture we used
model_architecture = "PPO"

## Define the commit message
commit_message = "trained model for Walker2d-v4 using PPO"

# method save, evaluate, generate a model card and record a replay video of your agent before pushing the repo to the hub
package_to_hub(model=model, # Our trained model
               model_name=model_name, # The name of our trained model 
               model_architecture=model_architecture, # The model architecture we used: in our case PPO
               env_id=env_id, # Name of the environment
               eval_env=eval_env, # Evaluation Environment
               repo_id=repo_id, # id of the model repository from the Hugging Face Hub (repo_id = {organization}/{repo_name} for instance Amankankriya/ppo-CarRacing-v2
               commit_message=commit_message)